# Telangana RERA Scraper

In [1]:
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import Chrome, Safari
from selenium.webdriver.support.wait import WebDriverWait
import platform
import os
import time

browser: WebDriver

if platform.system() == 'Darwin' and False:
	browser = Safari()
else:
    browser = Chrome(executable_path="/usr/local/bin/chromedriver")

browser.implicitly_wait(0.5)
browser.get("http://rerait.telangana.gov.in/SearchList/Search")
page_size = browser.find_element_by_id('PageSize')

# Uncomment this when not in development; it'll use 100 elements per page instead of the default 10
# page_size.send_keys("0")

counter: int = 1

2. Scrape the 100 entries on the page

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions

row_xpath = '//*[@id="gridview"]/div[1]/div/table/tbody/tr'

rows: list[WebElement] = browser.find_elements_by_xpath(row_xpath)
for row_i, row in enumerate(rows):
	cells: list[WebElement] = row.find_elements_by_xpath('{}[{}]/td'.format(row_xpath, row_i + 1))
	# Intentionally using camelCase instead of snake_case as the schema uses camelCase
	projectName = cells[1].get_attribute('innerHTML')
	promoterName = cells[2].get_attribute('innerHTML')
	lastModifiedDate = cells[3].get_attribute('innerHTML')
	view_link: WebElement = cells[4].find_element_by_xpath('./b').find_element_by_xpath('./a')
	view_link.click()
	browser.switch_to.window(browser.window_handles[1])
	WebDriverWait(browser, 10).until(expected_conditions.presence_of_element_located((By.ID, "DivPartial")))
	mandal = browser.find_element_by_xpath('//*[@id="DivProject"]/div[4]/div[2]/div[2]/div/div[2]')\
		.get_attribute('innerHTML').strip()
	pincode = browser.find_element_by_xpath('//*[@id="DivProject"]/div[4]/div[2]/div[4]/div/div[2]')\
		.get_attribute('innerHTML').strip()
	locality = browser.find_element_by_xpath('//*[@id="DivProject"]/div[4]/div[2]/div[3]/div[1]/div[4]')\
		.get_attribute('innerHTML').strip()

	browser.close()
	browser.switch_to.window(browser.window_handles[0])

3. Write the 100 entries to a file

In [ ]:
os.makedirs('out', exist_ok=True)
file = open("out/{}.csv".format(counter), "w")
# TODO
file.close()
counter += 1 # Use this in the file name